<div align="center">

  <a href="https://ultralytics.com/yolov5" target="_blank">
    <img width="1024", src="https://raw.githubusercontent.com/ultralytics/assets/main/yolov5/v70/splash.png"></a>

[中文](https://docs.ultralytics.com/zh/) | [한국어](https://docs.ultralytics.com/ko/) | [日本語](https://docs.ultralytics.com/ja/) | [Русский](https://docs.ultralytics.com/ru/) | [Deutsch](https://docs.ultralytics.com/de/) | [Français](https://docs.ultralytics.com/fr/) | [Español](https://docs.ultralytics.com/es/) | [Português](https://docs.ultralytics.com/pt/) | [العربية](https://docs.ultralytics.com/ar/)

  <a href="https://bit.ly/yolov5-paperspace-notebook"><img src="https://assets.paperspace.io/img/gradient-badge.svg" alt="Run on Gradient"></a>
  <a href="https://colab.research.google.com/github/ultralytics/yolov5/blob/master/tutorial.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"></a>
  <a href="https://www.kaggle.com/models/ultralytics/yolov5"><img src="https://kaggle.com/static/images/open-in-kaggle.svg" alt="Open In Kaggle"></a>

This <a href="https://github.com/ultralytics/yolov5">YOLOv5</a> 🚀 notebook by <a href="https://ultralytics.com">Ultralytics</a> presents simple train, validate and predict examples to help start your AI adventure.<br>We hope that the resources in this notebook will help you get the most out of YOLOv5. Please browse the YOLOv5 <a href="https://docs.ultralytics.com/yolov5">Docs</a> for details, raise an issue on <a href="https://github.com/ultralytics/yolov5">GitHub</a> for support, and join our <a href="https://ultralytics.com/discord">Discord</a> community for questions and discussions!

</div>

# Setup

Clone GitHub [repository](https://github.com/ultralytics/yolov5), install [dependencies](https://github.com/ultralytics/yolov5/blob/master/requirements.txt) and check PyTorch and GPU.

In [2]:
!git clone https://github.com/ultralytics/yolov5  # clone
%cd yolov5
%pip install -qr requirements.txt comet_ml  # install

import torch
import utils
display = utils.notebook_init()  # checks

YOLOv5 🚀 v7.0-408-g3fee72b5 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)


Setup complete ✅ (2 CPUs, 12.7 GB RAM, 41.3/112.6 GB disk)


In [3]:
from google.colab import drive
drive.mount('/content/drive')

# Verify files (replace with your path)
!ls "/content/drive/MyDrive/annotated_imgs"

Mounted at /content/drive
K1_msz.png  K3_msz.png	K4_msz.png  T9_msz.png	Z9_msz.png


In [6]:
from google.colab import drive
drive.mount('/content/drive')

import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from skimage.segmentation import watershed
from skimage.feature import peak_local_max
from scipy import ndimage

# Create directories if they don't exist
!mkdir -p "/content/drive/MyDrive/dataset/precise_labels"
!mkdir -p "/content/drive/MyDrive/dataset/visualizations"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
# ========== MOUNT DRIVE & IMPORTS ==========
from google.colab import drive
drive.mount('/content/drive')

import cv2
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from skimage.segmentation import watershed
from skimage.feature import peak_local_max
from scipy import ndimage
import re

# ========== CONFIGURATION ==========
ANNOTATED_DIR = Path("/content/drive/MyDrive/annotated_imgs")
ORIGINAL_DIR = Path("/content/drive/MyDrive/dataset/originals")
OUTPUT_DIR = Path("/content/drive/MyDrive/dataset/precise_labels")
VISUAL_DIR = Path("/content/drive/MyDrive/dataset/visualizations")

# ========== ENHANCED SEGMENTATION PARAMETERS ==========
INITIAL_HUE_TOLERANCE = 15    # Increased tolerance for color variation
INITIAL_SAT_TOLERANCE = 25    # More lenient saturation matching
GROWTH_STEPS = 5              # More growth iterations
GROWTH_FACTOR = 1.8           # Progressive tolerance relaxation
MIN_BACTERIA_AREA = 50        # pixels^2
MAX_BACTERIA_AREA = 2000      # pixels^2

# ========== UPDATED REGION GROWING WITH SAFE COORDINATE HANDLING ==========
def adaptive_region_growing(annot_img, orig_img):
    # Resize annotation to match original dimensions while preserving annotations
    if annot_img.shape != orig_img.shape:
        # First detect blue markers on original size
        annot_hsv = cv2.cvtColor(annot_img, cv2.COLOR_BGR2HSV)
        blue_mask = cv2.inRange(annot_hsv, np.array([90,70,70]), np.array([130,255,255]))
        contours, _ = cv2.findContours(blue_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # Calculate scaling factors
        scale_x = orig_img.shape[1] / annot_img.shape[1]
        scale_y = orig_img.shape[0] / annot_img.shape[0]

        # Scale all contour points
        scaled_contours = []
        for cnt in contours:
            scaled_cnt = cnt * np.array([scale_x, scale_y])
            scaled_contours.append(scaled_cnt.astype(np.int32))

        # Create new annotation image with correct size
        new_annot = np.zeros_like(orig_img)
        cv2.drawContours(new_annot, scaled_contours, -1, (255,0,0), -1)
        annot_img = new_annot

    # Rest of your processing remains the same...
    hsv = cv2.cvtColor(orig_img, cv2.COLOR_BGR2HSV)
    annot_hsv = cv2.cvtColor(annot_img, cv2.COLOR_BGR2HSV)

    # SAFE seed detection with boundary checks
    blue_mask = cv2.inRange(annot_hsv, np.array([90,70,70]), np.array([130,255,255]))
    seeds = []
    contours, _ = cv2.findContours(blue_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for cnt in contours:
        M = cv2.moments(cnt)
        if M["m00"] > 0:
            cX = min(max(int(M["m10"] / M["m00"]), 0), orig_img.shape[1]-1)
            cY = min(max(int(M["m01"] / M["m00"]), 0), orig_img.shape[0]-1)
            seeds.append((cX, cY))

    # Create background mask
    _, bg_mask = cv2.threshold(hsv[:,:,2], 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    bg_mask = (bg_mask == 0).astype(np.uint8)

    # Process each seed point
    final_mask = np.zeros_like(hsv[:,:,0])
    for seed in seeds:
        # Relocate seed to optimal position
        x, y = relocate_seed(seed, hsv, bg_mask, orig_img.shape)
        h_ref, s_ref, _ = hsv[y, x]

        # Multi-stage region growing
        region = np.zeros_like(final_mask)
        queue = [(x, y)]
        visited = set()

        for tolerance in np.linspace(1.0, GROWTH_FACTOR, GROWTH_STEPS):
            h_tol = INITIAL_HUE_TOLERANCE * tolerance
            s_tol = INITIAL_SAT_TOLERANCE * tolerance

            while queue:
                x, y = queue.pop(0)
                if (x, y) in visited:
                    continue
                if not (0 <= x < orig_img.shape[1] and 0 <= y < orig_img.shape[0]):
                    continue
                if bg_mask[y, x]:
                    continue

                # Color similarity check with overflow protection
                h, s, _ = hsv[y, x]
                if abs(int(h) - int(h_ref)) < h_tol and abs(int(s) - int(s_ref)) < s_tol:
                    region[y, x] = 255
                    visited.add((x, y))

                    # 8-way expansion
                    for dx in [-1, 0, 1]:
                        for dy in [-1, 0, 1]:
                            if dx == 0 and dy == 0:
                                continue
                            nx, ny = x + dx, y + dy
                            if (0 <= nx < orig_img.shape[1] and 0 <= ny < orig_img.shape[0]):
                                queue.append((nx, ny))

            # Morphological assistance in later stages
            if tolerance > GROWTH_FACTOR/2:
                region = cv2.dilate(region, np.ones((3,3), np.uint8))

        final_mask = cv2.bitwise_or(final_mask, region)

    # Post-processing
    final_mask = cv2.morphologyEx(final_mask, cv2.MORPH_CLOSE,
                                cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (7,7)))

    # Watershed for overlapping regions
    distance = ndimage.distance_transform_edt(final_mask)
    coords = peak_local_max(distance, min_distance=20, labels=final_mask)
    markers = np.zeros_like(final_mask, dtype=np.int32)

    for i, (x, y) in enumerate(coords):
        markers[y, x] = i + 1

    labels = watershed(-distance, markers, mask=final_mask)

    # Extract validated contours
    bacteria_contours = []
    for label in np.unique(labels):
        if label == 0:
            continue

        mask = np.zeros_like(labels, dtype=np.uint8)
        mask[labels == label] = 255

        cnts, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        for cnt in cnts:
            area = cv2.contourArea(cnt)
            if MIN_BACTERIA_AREA < area < MAX_BACTERIA_AREA:
                bacteria_contours.append(cnt)

    return bacteria_contours

def relocate_seed(original_seed, hsv_img, bg_mask, img_shape):
    x, y = original_seed
    best_score = -np.inf
    best_pos = (min(max(x, 0), img_shape[1]-1),
                min(max(y, 0), img_shape[0]-1))

    # Search in 15x15 window around original seed
    for dx in range(-7, 8):
        for dy in range(-7, 8):
            nx = min(max(x + dx, 0), img_shape[1]-1)
            ny = min(max(y + dy, 0), img_shape[0]-1)

            if bg_mask[ny, nx]:
                continue

            # Score based on local color variation
            window = hsv_img[max(0,ny-5):min(ny+6, img_shape[0]),
                           max(0,nx-5):min(nx+6, img_shape[1])]
            if window.size == 0:
                continue

            std_dev = np.std(window, axis=(0,1))
            score = std_dev[0] + std_dev[1]  # Hue + Saturation variation

            if score > best_score:
                best_score = score
                best_pos = (nx, ny)

    return best_pos

# ========== VISUALIZATION & OUTPUT ==========
def process_image_pair(annot_path, orig_path):
    # Load images
    annot_img = cv2.imread(str(annot_path))
    orig_img = cv2.imread(str(orig_path))

    if annot_img is None or orig_img is None:
        print(f"⚠️ Error loading {annot_path.name} or {orig_path.name}")
        return

    # Perform segmentation
    contours = adaptive_region_growing(annot_img, orig_img)

    # Create enhanced visualization
    display_img = orig_img.copy()

    # Draw filled contours with transparency
    overlay = display_img.copy()
    cv2.drawContours(overlay, contours, -1, (0,255,255), -1)  # Yellow fill
    cv2.addWeighted(overlay, 0.3, display_img, 0.7, 0, display_img)

    # Draw contour edges in contrasting color
    cv2.drawContours(display_img, contours, -1, (255,0,255), 2)  # Purple edges

    # Create comparison figure
    fig, ax = plt.subplots(1, 3, figsize=(20, 6))
    fig.suptitle(f"Analysis for {orig_path.name}", fontsize=16, y=0.95)

    # Original Image
    ax[0].imshow(cv2.cvtColor(orig_img, cv2.COLOR_BGR2RGB))
    ax[0].set_title("Original Image\n", fontsize=12)
    ax[0].axis('off')

    # Annotation Points
    blue_mask = cv2.inRange(cv2.cvtColor(annot_img, cv2.COLOR_BGR2HSV),
                           np.array([90,70,70]), np.array([130,255,255]))
    ax[1].imshow(blue_mask, cmap='Blues')
    ax[1].set_title(f"Annotation Points: {len(contours)}\n", fontsize=12)
    ax[1].axis('off')

    # Segmentation Result
    ax[2].imshow(cv2.cvtColor(display_img, cv2.COLOR_BGR2RGB))
    ax[2].set_title(f"Segmented Bacteria: {len(contours)}\n", fontsize=12)
    ax[2].axis('off')

    plt.savefig(VISUAL_DIR / f"{orig_path.stem}_seg.png", bbox_inches='tight', dpi=150)
    plt.close()

    # Save YOLO OBB format labels
    save_yolo_labels(orig_img.shape, contours, orig_path.stem)

def save_yolo_labels(img_shape, contours, stem):
    height, width = img_shape[:2]
    with open(OUTPUT_DIR / f"{stem}.txt", "w") as f:
        for cnt in contours:
            # Get rotated bounding box
            rect = cv2.minAreaRect(cnt)
            box = cv2.boxPoints(rect)
            box_norm = box / np.array([width, height])

            # YOLO OBB format: class x1 y1 x2 y2 x3 y3 x4 y4
            line = f"0 " + " ".join([f"{p[0]:.6f} {p[1]:.6f}" for p in box_norm])
            f.write(line + "\n")

# ========== GENERIC FILE HANDLING ==========
def find_matching_pairs(annotated_dir, original_dir):
    """Matches any files with pattern: <name>_msz.<ext> to <name>.<ext>"""
    pattern = re.compile(r'^(.+)_msz(\.[a-zA-Z]+)$', re.IGNORECASE)
    pairs = []

    for annot_path in annotated_dir.glob('*'):
        match = pattern.match(annot_path.name)
        if match:
            base_name = match.group(1) + match.group(2)
            orig_path = original_dir / base_name
            if orig_path.exists():
                pairs.append((annot_path, orig_path))
            else:
                print(f"⚠️ Missing original for {annot_path.name}")
    return pairs

# ========== MAIN EXECUTION ==========
def main():
    # Setup directories
    OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
    VISUAL_DIR.mkdir(parents=True, exist_ok=True)

    # Find all valid image pairs
    pairs = find_matching_pairs(ANNOTATED_DIR, ORIGINAL_DIR)

    if not pairs:
        print("❌ No valid image pairs found!")
        print("Ensure annotated files follow <name>_msz.<ext> pattern")
        return

    processed = 0
    for annot_path, orig_path in pairs:
        try:
            process_image_pair(annot_path, orig_path)
            processed += 1
            print(f"✅ Processed {orig_path.name}")
        except Exception as e:
            print(f"❌ Error processing {orig_path.name}: {str(e)}")

    print(f"\n🎉 Finished! Processed {processed}/{len(pairs)} image pairs")
    print(f"Labels saved to: {OUTPUT_DIR}")
    print(f"Visualizations saved to: {VISUAL_DIR}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
# Clear previous outputs (optional)
!rm -rf "/content/drive/MyDrive/dataset/precise_labels/*"
!rm -rf "/content/drive/MyDrive/dataset/visualizations/*"

# Execute
main()

❌ Error processing K4.png: index 1800 is out of bounds for axis 0 with size 1536
❌ Error processing T9.png: index 1978 is out of bounds for axis 0 with size 1536
❌ Error processing K1.png: index 1653 is out of bounds for axis 0 with size 1536
❌ Error processing Z9.png: index 1682 is out of bounds for axis 0 with size 1536
❌ Error processing K3.png: index 1797 is out of bounds for axis 0 with size 1536

🎉 Finished! Processed 0/5 image pairs
Labels saved to: /content/drive/MyDrive/dataset/precise_labels
Visualizations saved to: /content/drive/MyDrive/dataset/visualizations
